In [ ]:
from RIS_MIMO_environment import RIS_MIMO_env
# import helpers
import numpy as np
import random
import os
import torch
# Import stable baselines stuff
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.noise import NormalActionNoise

In [ ]:
env_args = {"ntx_antennas":8,
        "nrx_antennas":2,
        "n_users":4,
        "nris_surfaces":32,
        "nris_elements":1}

In [ ]:
env = RIS_MIMO_env(**env_args)

In [ ]:
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    score = 0
    action = env.action_space.sample()
    obs, reward, done ,info = env.step(action)
    score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
action_noise = NormalActionNoise(mean=np.zeros(env.action_space.shape), sigma=0.1 * np.ones(env.action_space.shape))
model = DDPG(
    policy='MlpPolicy', 
    env=env,
    buffer_size=100000,
    learning_rate= 0.001,
    batch_size=16,
    tau=0.001,
    action_noise=action_noise,
    verbose=1)
model.learn(total_timesteps=1000)
model_name = 'ppoRISMIMO-32e-1ris-4u'
ris_miso_models = os.path.join('RIS_MIMO_models',model_name)
model.save(model_name)

In [ ]:
eval_env = Monitor(env)
#model = PPO.load(shower_path,env)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward = {mean_reward:.2f} +/- {std_reward}")